In [19]:
from poseidon import poseidon
import time
import hashlib
import pandas as pd
import random
from sklearn.externals import joblib
import json
millis = int(round(time.time() * 1000))
#授权应用的accessKey
accessKey = 'a3358652-483d-4e6a-973e-f0e49d445a4e'
#授权应用的secretKey
secretKey = 'c8585cde-85c3-4735-97a1-18141366c4df'
#token将在get_token方法中通过APIM接口进行获取
token = ''
#IAM地址
base = 'https://beta-apim-cn4.eniot.io/data-federation/v2.0/channels/read/'
#base = 'https://beta-portal-cn4.eniot.io:8081/data-federation/v2.0/channels/read/'
#访问通道ID
channel = 'ch-4cb5c2'
#组织ID
orgId = 'o15632609593521'
url = base + channel + '?orgId=' + orgId


#获取通道下的schema，可以理解为mysql中的show databases
def show_schema():
    header={'Content-Type':'application/json','Authorization':'Bearer ' + token}
    #data = {"sqlQuery": 'show schemas', "queue": None, "itemFormat": None}
    data = {"sqlQuery": 'show databases', "queue": None, "itemFormat": None}
    req = poseidon.urlopen(accessKey, secretKey, url, data, header)
    print(req)

#获取表，可以理解为mysql中的show tables
def query_hive_tables():
    header={'Content-Type':'application/json','Authorization':'Bearer ' + token}
    #data = {"sqlQuery": 'show tables from `hive.db_jinghui_zhao`', "queue": None, "itemFormat": None}
    data = {"sqlQuery": 'show tables from `df_hive.data_o15632609593521`', "queue": None, "itemFormat": None}
    req = poseidon.urlopen(accessKey, secretKey, url, data, header)
    print(pd.DataFrame(req.get('data').get('rows'))[['TABLE_NAME']])

#查询某个表的数据
def query_hive_data(masterid):
    header={'Content-Type':'application/json','Authorization':'Bearer ' + token}
    #data = {"sqlQuery": 'select * from `df_hive.data_o15632609593521.iris_tbl`', "queue": None, "itemFormat": None}
    data = {"sqlQuery": f'select * from df_hive.data_o15632609593521.`kmmlds1` where masterid=\'{masterid}\'', "queue": None, "itemFormat": None}
    #data = {"sqlQuery": 'select * from df_hive.`iris_tbl`', "queue": None, "itemFormat": None}
    req = poseidon.urlopen(accessKey, secretKey, url, data, header)
    dataframe = pd.DataFrame(req.get('data').get('rows'))
    #print(dataframe)
    #print(req)
    return dataframe

#获取应用Token
def get_token():
    global token
    url = 'https://beta-apim-cn4.eniot.io/apim-token-service/v2.0/token/get'
    hash = hashlib.sha256()
    hash.update((accessKey+str(millis)+secretKey).encode('utf-8'))
    data = {"appKey": accessKey, "encryption": hash.hexdigest(), "timestamp":millis}
    req = poseidon.urlopen(accessKey, secretKey, url, data)
    token = req.get('data').get('accessToken')
    print(token)

In [20]:
# Model Tranining Sample
def train(in_alpha, in_l1_ratio, masterid):
    import os
    import warnings
    import sys

    import pandas as pd
    import numpy as np
    from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
    from sklearn.model_selection import train_test_split
    from sklearn.linear_model import ElasticNet

    import mlflow
    import mlflow.sklearn
    
    import logging
    logging.basicConfig(level=logging.WARN)
    logger = logging.getLogger(__name__)

    def eval_metrics(actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2


    warnings.filterwarnings("ignore")
    np.random.seed(40)

    # Read data from hive table prepared before
    try:
        data=query_hive_data(masterid)[["x_basic_hour","x_basic_horizon","i_set","ec_ws","ec_wd","ec_tmp","ec_press","ec_rho","ec_dist","gfs_ws","gfs_wd","gfs_tmp","gfs_press","gfs_rho","gfs_dist","speed","power"]]
        data =data.fillna('1')
        #print(data)
    except Exception as e:
        logger.exception(
            "Get Hive Data Error: %s", e)

    # Split the data into training and test sets. (0.75, 0.25) split.
    train, test = train_test_split(data)

    # The predicted column is "power"
    train_x = train.drop(["power","speed"], axis=1)
    test_x = test.drop(["power","speed"], axis=1)
    train_y = train[["power"]]
    test_y = test[["power"]]
    
    
    # Set default values if no alpha is provided
    if float(in_alpha) is None:
        alpha = 0.5
    else:
        alpha = float(in_alpha)

    # Set default values if no l1_ratio is provided
    if float(in_l1_ratio) is None:
        l1_ratio = 0.5
    else:
        l1_ratio = float(in_l1_ratio)

    # Useful for multiple runs (only doing one run in this sample notebook)    
    with mlflow.start_run():
        # Execute ElasticNet
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(train_x, train_y)
        
        #print(test_x)
        # Evaluate Metrics
        predicted_powers = lr.predict(test_x)
        (rmse, mae, r2) = eval_metrics(test_y, predicted_powers)

        # Print out metrics
        print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        # Log parameter, metrics, and model to MLflow
        mlflow.log_param("alpha", alpha)
        mlflow.log_param("l1_ratio", l1_ratio)
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mae", mae)

        mlflow.sklearn.log_model(lr, "model")
        
        

        #joblib.dump(lr, 'kongmingml.pkl')


In [21]:
if __name__ == '__main__':
    get_token()
    #show_schema()
    #query_hive_tables()
    #query_hive_data()
    #join_hive_data()
    #create_file()   
    train(0.5,0.5,'CGNWF0046')

f9aebdfe360e355bf75a69cc730ca40a3c47a6ce9c50b9a02da0c42f625d0ada
Elasticnet model (alpha=0.500000, l1_ratio=0.500000):
  RMSE: 15069.379176201654
  MAE: 11268.739369488349
  R2: 0.6629556410296898


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-21-ebcad70b3a0f>", line 8, in <module>
    train(0.5,0.5,'CGNWF0046')
  File "<ipython-input-20-e22850eb13ed>", line 85, in train
    mlflow.sklearn.log_model(lr, "model")
  File "/opt/conda/lib/python3.7/site-packages/mlflow/sklearn.py", line 200, in log_model
    registered_model_name=registered_model_name)
  File "/opt/conda/lib/python3.7/site-packages/mlflow/models/__init__.py", line 118, in log
    registered_model_name)
  File "/opt/conda/lib/python3.7/site-packages/mlflow/utils/file_utils.py", line 191, in __exit__
    assert os.path.exists(os.getcwd())
FileNotFoundError: [Errno 2] No such file or directory

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/IPython/c

FileNotFoundError: [Errno 2] No such file or directory

In [105]:
# import json
# import pandas as pandas
# raw_data='''{
# "data": {
# "X_basic.time": ["2019-07-01 00:00:00", "2019-07-01 01:00:00", "2019-07-01 02:00:00", "2019-07-01 03:00:00", "2019-07-01 04:00:00", "2019-07-01 05:00:00", "2019-07-01 06:00:00", "2019-07-01 07:00:00", "2019-07-01 08:00:00", "2019-07-01 09:00:00", "2019-07-01 10:00:00", "2019-07-01 11:00:00", "2019-07-01 12:00:00", "2019-07-01 13:00:00", "2019-07-01 14:00:00", "2019-07-01 15:00:00", "2019-07-01 16:00:00", "2019-07-01 17:00:00", "2019-07-01 18:00:00", "2019-07-01 19:00:00", "2019-07-01 20:00:00", "2019-07-01 21:00:00", "2019-07-01 22:00:00", "2019-07-01 23:00:00", "2019-07-02 00:00:00", "2019-07-02 01:00:00", "2019-07-02 02:00:00", "2019-07-02 03:00:00", "2019-07-02 04:00:00", "2019-07-02 05:00:00", "2019-07-02 06:00:00", "2019-07-02 07:00:00", "2019-07-02 08:00:00", "2019-07-02 09:00:00", "2019-07-02 10:00:00", "2019-07-02 11:00:00", "2019-07-02 12:00:00", "2019-07-02 13:00:00", "2019-07-02 14:00:00", "2019-07-02 15:00:00", "2019-07-02 16:00:00", "2019-07-02 17:00:00", "2019-07-02 18:00:00", "2019-07-02 19:00:00", "2019-07-02 20:00:00", "2019-07-02 21:00:00", "2019-07-02 22:00:00", "2019-07-02 23:00:00", "2019-07-03 00:00:00"], 
# "X_basic.hour": [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 0], 
# "X_basic.forecast_time": ["2019-07-01 00:00:00", "2019-07-01 00:00:00", "2019-07-01 00:00:00", "2019-07-01 00:00:00", "2019-07-01 00:00:00", "2019-07-01 00:00:00", "2019-07-01 00:00:00", "2019-07-01 00:00:00", "2019-07-01 00:00:00", "2019-07-01 00:00:00", "2019-07-01 00:00:00", "2019-07-01 00:00:00", "2019-07-01 00:00:00", "2019-07-01 00:00:00", "2019-07-01 00:00:00", "2019-07-01 00:00:00", "2019-07-01 00:00:00", "2019-07-01 00:00:00", "2019-07-01 00:00:00", "2019-07-01 00:00:00", "2019-07-01 00:00:00", "2019-07-01 00:00:00", "2019-07-01 00:00:00", "2019-07-01 00:00:00", "2019-07-01 00:00:00", "2019-07-01 00:00:00", "2019-07-01 00:00:00", "2019-07-01 00:00:00", "2019-07-01 00:00:00", "2019-07-01 00:00:00", "2019-07-01 00:00:00", "2019-07-01 00:00:00", "2019-07-01 00:00:00", "2019-07-01 00:00:00", "2019-07-01 00:00:00", "2019-07-01 00:00:00", "2019-07-01 00:00:00", "2019-07-01 00:00:00", "2019-07-01 00:00:00", "2019-07-01 00:00:00", "2019-07-01 00:00:00", "2019-07-01 00:00:00", "2019-07-01 00:00:00", "2019-07-01 00:00:00", "2019-07-01 00:00:00", "2019-07-01 00:00:00", "2019-07-01 00:00:00", "2019-07-01 00:00:00", "2019-07-01 00:00:00"], 
# "X_basic.horizon": [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48], 
# "i.set": [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
# "EC.nwp_time": ["2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00"], 
# "EC.ws": [1.33, 1.3009334, 1.2718667, 1.2428, 1.3562, 1.4696, 1.583, 1.2687299, 0.95445997, 0.64019, 0.90312666, 1.1660633, 1.429, 1.4888667, 1.5487334, 1.6086, 1.7422334, 1.8758667, 2.0095, 2.1807334, 2.3519666, 2.5232, 2.4561667, 2.3891332, 2.3221, 2.2458, 2.1694999, 2.0932, 2.4462667, 2.7993333, 3.1524, 3.4846332, 3.8168666, 4.1491, 3.8212333, 3.4933665, 3.1655, 2.8962667, 2.6270332, 2.3578, 2.3612332, 2.3646667, 2.3681, 2.4056332, 2.4431667, 2.4807, 2.4635334, 2.4463665, 2.4292], 
# "EC.wd": [241.20001, 244.10333, 247.00665, 249.91003, 262.69666, 275.48334, 288.27002, 276.7533, 265.2367, 253.71997, 246.79663, 239.87335, 232.95001, 237.66998, 242.39001, 247.10999, 249.99335, 252.87665, 255.76001, 254.23334, 252.70667, 251.18, 256.39, 261.59998, 266.81, 273.52002, 280.22998, 286.94, 284.06665, 281.19336, 278.32, 271.89, 265.45996, 259.03003, 253.7367, 248.44336, 243.15002, 246.29999, 249.45001, 252.59998, 256.22998, 259.86, 263.49, 270.76666, 278.04333, 285.32, 286.40332, 287.4867, 288.57], 
# "EC.tmp": [18.327, 19.069666, 19.812334, 20.555, 21.062666, 21.570333, 22.078, 22.091333, 22.104666, 22.118, 21.030666, 19.943335, 18.856, 18.546667, 18.237333, 17.928, 17.848333, 17.768665, 17.689, 17.607666, 17.526333, 17.445, 17.688, 17.931, 18.174, 19.049334, 19.924665, 20.8, 21.622665, 22.445333, 23.268, 23.274666, 23.281334, 23.288, 22.343, 21.397999, 20.453, 19.903334, 19.353666, 18.804, 18.582666, 18.361334, 18.14, 18.050999, 17.962, 17.873, 18.131666, 18.390333, 18.649], 
# "EC.pres": [829.72, 830.0333, 830.3466, 830.66, 830.32666, 829.9933, 829.66, 829.23663, 828.81335, 828.39, 828.69, 828.99, 829.29, 829.99335, 830.69666, 831.4, 831.04004, 830.68, 830.32, 830.2733, 830.2267, 830.18, 830.5867, 830.99335, 831.4, 831.51666, 831.63336, 831.75, 831.1767, 830.60333, 830.03, 829.4267, 828.8233, 828.22, 828.4933, 828.76666, 829.04, 829.74664, 830.4533, 831.16, 830.8566, 830.55334, 830.25, 829.95, 829.64996, 829.35, 829.57996, 829.81, 830.04], 
# "EC.rho": [0.9916758030082774, 0.9895289962508731, 0.9873930671580704, 0.9852680652882487, 0.983173277132302, 0.9810856786972699, 0.979005330631279, 0.9784615617058195, 0.9779179480821056, 0.9773742976532892, 0.9813420761262862, 0.9853392842857234, 0.9893662744840102, 0.991255465204745, 0.9931486225919256, 0.9950458362526601, 0.9948873218067945, 0.9947286281640354, 0.9945698852346762, 0.9947921438779024, 0.995014643326736, 0.9952371474417167, 0.9948927633064221, 0.9945488943271201, 0.9942055990057524, 0.9913663695347825, 0.9885441577361368, 0.985738669508786, 0.9823100815647162, 0.9789004852464784, 0.9755098649752867, 0.9747789028879918, 0.9740478495821158, 0.9733169532503687, 0.9767518718933428, 0.9802089053655669, 0.9836881622667665, 0.9863732471529474, 0.9890684541183578, 0.9917738506662764, 0.9921639934748434, 0.9925548892236702, 0.992946290144862, 0.9928908707045153, 0.9928353626889924, 0.99277991989915, 0.9921733354396797, 0.9915679195447455, 0.9909635292440456], 
# "EC.dist": [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60], 
# "GFS.nwp_time": ["2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00", "2019-06-30 12:00:00"], 
# "GFS.ws": [1.7459, 1.7463, 1.3237, 1.0615, 0.50111, 0.45386, 0.25424, 0.37834, 0.65534, 0.72903, 0.90213, 0.77058, 0.40455, 0.63919, 0.73106, 0.51279, 0.98015, 0.6217, 0.77995, 1.2492, 1.3586, 1.4119, 1.577, 1.505, 1.2436, 1.3133, 1.6796, 1.9819, 1.8584, 1.3959, 1.3728, 0.89585, 1.4478, 1.7458, 1.9353, 1.9419, 2.2374, 2.4431, 2.1857, 1.9936, 1.8383, 1.7509, 1.6839, 1.5973, 1.6528, 1.6963, 1.6198, 1.2812, 1.1869], 
# "GFS.wd": [186.76001, 191.71997, 199.58002, 211.28003, 238.72998, 244.10999, 319.52002, 61.001007, 47.88199, 49.009003, 359.72998, 346.96997, 268.88, 229.53998, 280.20996, 287.91003, 304.51, 303.57996, 278.15002, 282.63, 295.57996, 297.41003, 308.07996, 306.68, 293.34998, 275.91003, 267.93, 268.43, 268.91003, 273.71997, 265.57, 278.63, 261.32996, 247.66998, 233.12, 219.79999, 212.69, 222.63, 226.08997, 228.68, 226.87, 234.85999, 240.84998, 247.84998, 263.53003, 273.81, 288.95996, 287.25, 290.66003], 
# "GFS.tmp": [18.651, 18.831, 18.87, 18.97, 19.151, 19.451, 19.65, 19.651, 19.751, 19.751, 19.75, 19.35, 19.05, 18.799, 18.75, 18.678, 18.35, 17.964, 17.85, 17.85, 17.45, 17.15, 16.95, 16.863, 18.65, 20.75, 22.65, 24.15, 24.15, 24.95, 25.05, 25.55, 24.75, 23.85, 23.25, 22.45, 20.586, 19.656, 19.15, 18.844, 18.75, 18.45, 18.15, 18.163, 18.031, 17.883, 17.916, 18.05, 18.898], 
# "GFS.pres": [843.09, 843.41, 843.77, 844.14, 843.97, 843.59, 843.05, 842.39, 841.85, 841.56, 841.37, 841.3, 841.81, 842.36, 843.14, 843.93, 843.97, 843.57, 843.32, 842.73, 842.3, 842.07, 842.32, 842.83, 843.29, 843.68, 844.07, 843.78, 843.25, 842.58, 841.79, 841.01, 840.44, 840.14, 840.12, 840.42, 841.05, 841.82, 842.37, 842.98, 843.15, 842.71, 842.35, 841.94, 841.68, 841.36, 841.45, 841.66, 841.99], 
# "GFS.rho": [1.0065366916783614, 1.0062979851595728, 1.0065930610775693, 1.0066897271686468, 1.0058637499312235, 1.0043800221133028, 1.0030549122741894, 1.0022662259135926, 1.0012817732859702, 1.0009368523211273, 1.0007142861815448, 1.0019994134259942, 1.0036362006251849, 1.0051553595758493, 1.0062549905331664, 1.0074463215116152, 1.008627718588839, 1.0094864260155714, 1.0095826068114382, 1.008876286864065, 1.0097494831072855, 1.0105169630633353, 1.0115138477016707, 1.0124299138791377, 1.006778915151516, 1.000047472858534, 0.9940832221265518, 0.9887278486697294, 0.9881068031841823, 0.9846720690132904, 0.9834189483413867, 0.9808630757669228, 0.9828305775568036, 0.9854569617677089, 0.9874283070773688, 0.990454200361618, 0.9974866396564767, 1.001570943000887, 1.003960266118537, 1.0057401601348364, 1.0062669251465228, 1.0067765159470696, 1.007382829931929, 1.0068475697960015, 1.0069929341041324, 1.007121978294337, 1.0071155137724241, 1.0069033039769844, 1.0043732696959962], 
# "GFS.dist": [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60]}, 
# "pc": {
# "speed": [0.0, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0, 5.5, 6.0, 6.5, 7.0, 7.5, 8.0, 8.5, 9.0, 9.5, 10.0, 10.5, 11.0, 11.5, 12.0, 12.5, 13.0, 13.5, 14.0, 14.5, 15.0, 15.5, 16.0, 16.5, 17.0, 17.5, 18.0, 18.5, 19.0, 19.5, 20.0, 20.5, 21.0, 21.5, 22.0, 22.5, 23.0, 23.5, 24.0, 24.5, 25.0], 
# "power": [0.0, 0.0, 0.0, 0.0, 9.3, 27.3, 43.4, 1035.0, 3421.6, 5390.4, 8011.0, 11281.2, 15270.4, 19975.6, 25393.5, 31536.8, 38434.0, 46121.3, 54608.4, 63826.7, 73566.4, 83436.1, 92886.4, 101326.2, 108294.9, 113594.7, 117315.7, 119742.3, 121225.8, 122087.5, 122566.0, 122826.1, 122965.1, 123039.3, 123080.5, 123101.8, 123114.4, 123122.1, 123126.9, 123129.9, 123131.9, 123133.4, 123134.7, 123135.4, 123136.0, 123136.4, 123136.8, 123137.1, 123137.3, 123137.5, 123137.6]}}
# '''
# #[['X_basic.hour','X_basic.horizon','i.set','EC.ws','EC.wd','EC.tmp','EC.press','EC.rho','EC.dist','GFS.ws','GFS.wd','GFS.tmp','GFS.pres','GFS.rho','GFS.dist']])
# df_input1=pd.DataFrame(json.loads(raw_data).get('data'))[['X_basic.hour','X_basic.horizon','i.set','EC.ws','EC.wd','EC.tmp','EC.pres','EC.rho','EC.dist','GFS.ws','GFS.wd','GFS.tmp','GFS.pres','GFS.rho','GFS.dist']]


In [13]:
lr=joblib.load('kongmingml.pkl')
inputstr='''
{
  "data": {
    "names": [
      "X-basic.hour",
      "X-basic.horizon",
      "i-set",
      "EC-ws",
      "EC-wd",
      "EC-tmp",
      "EC-pres",
      "EC-rho",
      "EC-dist",
      "GFS-ws",
      "GFS-wd",
      "GFS-tmp",
      "GFS-pres",
      "GFS-rho",
      "GFS-dist"
    ],
    "ndarray": [
      [
        4,
        2,
        1,
        1.588,
        276.814,
        18.328,
        899.237,
        0,
        20,
        1.011,
        48.965,
        19.678,
        866.937,
        1,
        66
      ]
    ]
  }
}
'''
data=json.loads(inputstr).get('data')

#print(data.get('names'))
#print(data.get('ndarray'))

df1=pd.DataFrame(data.get('ndarray'),columns=data.get('names'))
print(df1)

   X-basic.hour  X-basic.horizon  i-set  EC-ws    EC-wd  EC-tmp  EC-pres  \
0             4                2      1  1.588  276.814  18.328  899.237   

   EC-rho  EC-dist  GFS-ws  GFS-wd  GFS-tmp  GFS-pres  GFS-rho  GFS-dist  
0       0       20   1.011  48.965   19.678   866.937        1        66  


In [15]:
predicted_powers = lr.predict(df1)
print(predicted_powers)

[90992.08078157]
